# `Промышленное машинное обучение на Spark`
## `Занятие 04: Детали SQL и Spark`

О чём можно узнать из этого ноутбука:

* RDD API
* Pivot/Unpivot
* Window function
* UDF
* Формат Parquet

Для эффективной работы UDF необходимо установить библиотеку, реализующую передачу данных между Spark и Python в Arrow формате:

In [1]:
! pip3 install pyspark pyarrow

In [2]:
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

from pyspark import SparkConf, SparkContext

conf = (
    SparkConf()
        .setMaster('local[*]')
)
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

### `Практика DataFrame`

In [3]:
!head hr_data.csv

Age,MonthlyIncome,Department,Gender,Education,Attrition
41,5993,Sales,Female,2,Yes
49,5130,Research & Development,Male,1,No
37,2090,Research & Development,Male,2,Yes
33,2909,Research & Development,Female,4,No
27,3468,Research & Development,Male,1,No
32,3068,Research & Development,Male,2,No
59,2670,Research & Development,Female,3,No
30,2693,Research & Development,Male,1,No
38,9526,Research & Development,Male,3,No


In [91]:
# Чтение CSV с заголовками
df = spark.read.csv("hr_data.csv", header=True, inferSchema=True)

# Вывести 5 строк
df.show(5)

+---+-------------+--------------------+------+---------+---------+
|Age|MonthlyIncome|          Department|Gender|Education|Attrition|
+---+-------------+--------------------+------+---------+---------+
| 41|         5993|               Sales|Female|        2|      Yes|
| 49|         5130|Research & Develo...|  Male|        1|       No|
| 37|         2090|Research & Develo...|  Male|        2|      Yes|
| 33|         2909|Research & Develo...|Female|        4|       No|
| 27|         3468|Research & Develo...|  Male|        1|       No|
+---+-------------+--------------------+------+---------+---------+
only showing top 5 rows



In [5]:
df.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- MonthlyIncome: integer (nullable = true)
 |-- Department: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Education: integer (nullable = true)
 |-- Attrition: string (nullable = true)



In [13]:
# 1) Посчитать количество девушек и парней (Gender, count)
(
    df.filter("Gender is not null").groupBy("Gender").agg(F.count("*").alias("cnt")).show()
)

+------+---+
|Gender|cnt|
+------+---+
|Female|588|
|  Male|882|
+------+---+



In [21]:
# 2) Сумма всех зарплат, средняя зарплата, минимальная зарплата и максимальная зарплата девушек и парней
# (Gender, cnt, sum_income, avg_income, min_income, max_income)
(
    df.groupBy("Gender").agg(
        F.count("MonthlyIncome").alias("cnt"),
        F.count_distinct("MonthlyIncome").alias("cnt_dist"),
        F.sum("MonthlyIncome").alias("sum_income"),
        F.round(F.avg("MonthlyIncome"), 3).alias("avg_income"),
        F.min("MonthlyIncome").alias("min_income"),
        F.max("MonthlyIncome").alias("max_income")
    ).show()
)

+------+---+--------+----------+----------+----------+----------+
|Gender|cnt|cnt_dist|sum_income|avg_income|min_income|max_income|
+------+---+--------+----------+----------+----------+----------+
|Female|588|     564|   3931701|  6686.566|      1129|     19973|
|  Male|882|     834|   5627608|  6380.508|      1009|     19999|
+------+---+--------+----------+----------+----------+----------+



In [30]:
# 3) Количество людей, сумма всех зарплат, средняя зарплата, минимальная и максимальные зарплаты в департаментах
# (Department, cnt, sum_income, avg_income, min_income, max_income)
(
    df.groupBy("Department").agg(
        F.count("*").alias("cnt"),
        F.sum("MonthlyIncome").alias("sum_income"),
        F.round(F.avg("MonthlyIncome"), 2).alias("avg_income"),
        F.min("MonthlyIncome").alias("min_income"),
        F.max("MonthlyIncome").alias("max_income")
    ).orderBy(F.col("avg_income")).show(truncate=False)
)

+----------------------+---+----------+----------+----------+----------+
|Department            |cnt|sum_income|avg_income|min_income|max_income|
+----------------------+---+----------+----------+----------+----------+
|Research & Development|961|6036284   |6281.25   |1009      |19999     |
|Human Resources       |63 |419234    |6654.51   |1555      |19717     |
|Sales                 |446|3103791   |6959.17   |1052      |19847     |
+----------------------+---+----------+----------+----------+----------+



In [33]:
# 4) Количество сотрудников по уровню образования
# (Education, employee_count)
(
    df.groupBy("Education").agg(
        F.count("*").alias("cnt")
    ).orderBy("Education").show()
)

+---------+---+
|Education|cnt|
+---------+---+
|        1|170|
|        2|282|
|        3|572|
|        4|398|
|        5| 48|
+---------+---+



In [41]:
# 5) Процент увольнений (Attrition) по департаментам
# (Department, total_employees, attrition_count, attrition_rate)
(
    df.groupBy("Department").agg(
        F.count("*").alias("total_employees"),
        F.sum(F.when(F.col("Attrition") == "Yes", 1).otherwise(0)).alias("attrition_count"),
        F.round(F.avg(F.when(F.col("Attrition") == "Yes", 1).otherwise(0))*100, 2).alias("attrition_rate")
    ).show()
)

+--------------------+---------------+---------------+--------------+
|          Department|total_employees|attrition_count|attrition_rate|
+--------------------+---------------+---------------+--------------+
|               Sales|            446|             92|         20.63|
|Research & Develo...|            961|            133|         13.84|
|     Human Resources|             63|             12|         19.05|
+--------------------+---------------+---------------+--------------+



In [58]:
# 6) Средний возраст сотрудников по гендеру (Gender, avg_age)
(
  df.groupBy("Gender").agg(
      F.round(F.avg("Age"), 1).alias("avg_age"),
      F.lit(0.0).alias("True?")
  ).show()
)

+------+-------+-----+
|Gender|avg_age|True?|
+------+-------+-----+
|Female|   37.3|  0.0|
|  Male|   36.7|  0.0|
+------+-------+-----+



In [50]:
df.filter("Gender = 'Female'").groupBy("Department").count().show()

+--------------------+-----+
|          Department|count|
+--------------------+-----+
|               Sales|  189|
|Research & Develo...|  379|
|     Human Resources|   20|
+--------------------+-----+



In [45]:
# 7) Соотношение мужчин и женщин в департаментах
# (Department, male_count, female_count)
(
    df.groupBy("Department").agg(
        F.count("*").alias("cnt"),
        F.sum(F.when(F.col("Gender") == "Male", 1).otherwise(0)).alias("male_count"),
        F.sum(F.when(F.col("Gender") == "Female", 1).otherwise(0)).alias("female_count")
    ).show(truncate=False)
)

+----------------------+---+----------+------------+
|Department            |cnt|male_count|female_count|
+----------------------+---+----------+------------+
|Sales                 |446|257       |189         |
|Research & Development|961|582       |379         |
|Human Resources       |63 |43        |20          |
+----------------------+---+----------+------------+



In [54]:
# 8) Возрастающий тренд средней зарплаты в зависимости от возраста (Age, avg_income)
(
    df.groupBy("Age").agg(
        F.round(F.avg("MonthlyIncome"), 1).alias("avg_income")
    ).orderBy("Age").show()
)

+---+----------+
|Age|avg_income|
+---+----------+
| 18|    1518.4|
| 19|    2075.0|
| 20|    2539.4|
| 21|    2491.5|
| 22|    2950.0|
| 23|    2515.4|
| 24|    3243.9|
| 25|    4144.8|
| 26|    3531.5|
| 27|    4190.4|
| 28|    3904.8|
| 29|    4584.4|
| 30|    4814.9|
| 31|    5525.4|
| 32|    5092.5|
| 33|    5500.5|
| 34|    5269.2|
| 35|    5425.3|
| 36|    5590.9|
| 37|    5853.9|
+---+----------+
only showing top 20 rows



### `Практика SQL`

In [92]:
df.show()

+---+-------------+--------------------+------+---------+---------+
|Age|MonthlyIncome|          Department|Gender|Education|Attrition|
+---+-------------+--------------------+------+---------+---------+
| 41|         5993|               Sales|Female|        2|      Yes|
| 49|         5130|Research & Develo...|  Male|        1|       No|
| 37|         2090|Research & Develo...|  Male|        2|      Yes|
| 33|         2909|Research & Develo...|Female|        4|       No|
| 27|         3468|Research & Develo...|  Male|        1|       No|
| 32|         3068|Research & Develo...|  Male|        2|       No|
| 59|         2670|Research & Develo...|Female|        3|       No|
| 30|         2693|Research & Develo...|  Male|        1|       No|
| 38|         9526|Research & Develo...|  Male|        3|       No|
| 36|         5237|Research & Develo...|  Male|        3|       No|
| 35|         2426|Research & Develo...|  Male|        3|       No|
| 29|         4193|Research & Develo...|Female| 

In [93]:
df.createOrReplaceTempView("df")

In [96]:
spark.sql("select * from df").show(5)

+---+-------------+--------------------+------+---------+---------+
|Age|MonthlyIncome|          Department|Gender|Education|Attrition|
+---+-------------+--------------------+------+---------+---------+
| 41|         5993|               Sales|Female|        2|      Yes|
| 49|         5130|Research & Develo...|  Male|        1|       No|
| 37|         2090|Research & Develo...|  Male|        2|      Yes|
| 33|         2909|Research & Develo...|Female|        4|       No|
| 27|         3468|Research & Develo...|  Male|        1|       No|
+---+-------------+--------------------+------+---------+---------+
only showing top 5 rows



In [98]:
# 1) Посчитать количество девушек и парней (Gender, count)
spark.sql("""
  select
    Gender,
    count(*) as cnt
  from df
  group by Gender
""").show()

+------+---+
|Gender|cnt|
+------+---+
|Female|588|
|  Male|882|
+------+---+



In [101]:
# 2) Сумма всех зарплат, средняя зарплата, минимальная зарплата и максимальная зарплата девушек и парней
# (Gender, sum_income, avg_income, min_income, max_income)
spark.sql("""
  select
    Gender,
    sum(MonthlyIncome) as sum_income,
    round(avg(MonthlyIncome), 2) as avg_income,
    min(MonthlyIncome) as min_income,
    max(MonthlyIncome) as max_income
  from df
  group by Gender
""").show()

+------+----------+----------+----------+----------+
|Gender|sum_income|avg_income|min_income|max_income|
+------+----------+----------+----------+----------+
|Female|   3931701|   6686.57|      1129|     19973|
|  Male|   5627608|   6380.51|      1009|     19999|
+------+----------+----------+----------+----------+



In [103]:
df.show(3)

+---+-------------+--------------------+------+---------+---------+
|Age|MonthlyIncome|          Department|Gender|Education|Attrition|
+---+-------------+--------------------+------+---------+---------+
| 41|         5993|               Sales|Female|        2|      Yes|
| 49|         5130|Research & Develo...|  Male|        1|       No|
| 37|         2090|Research & Develo...|  Male|        2|      Yes|
+---+-------------+--------------------+------+---------+---------+
only showing top 3 rows



In [105]:
# 3) Количество людей, сумма всех зарплат, средняя зарплата, минимальная и максимальные зарплаты в департаментах
# (Department, cnt, sum_income, avg_income, min_income, max_income)
spark.sql("""
  select
    Department,
    count(*) as cnt,
    sum(MonthlyIncome) as sum_income,
    round(avg(MonthlyIncome), 2) as avg_income,
    min(MonthlyIncome) as min_income,
    max(MonthlyIncome) as max_income
  from df
  group by Department
""").show()

+--------------------+---+----------+----------+----------+----------+
|          Department|cnt|sum_income|avg_income|min_income|max_income|
+--------------------+---+----------+----------+----------+----------+
|               Sales|446|   3103791|   6959.17|      1052|     19847|
|Research & Develo...|961|   6036284|   6281.25|      1009|     19999|
|     Human Resources| 63|    419234|   6654.51|      1555|     19717|
+--------------------+---+----------+----------+----------+----------+



In [106]:
df.show(3)

+---+-------------+--------------------+------+---------+---------+
|Age|MonthlyIncome|          Department|Gender|Education|Attrition|
+---+-------------+--------------------+------+---------+---------+
| 41|         5993|               Sales|Female|        2|      Yes|
| 49|         5130|Research & Develo...|  Male|        1|       No|
| 37|         2090|Research & Develo...|  Male|        2|      Yes|
+---+-------------+--------------------+------+---------+---------+
only showing top 3 rows



In [109]:
# 4) Количество сотрудников по уровню образования
# (Education, employee_count)
spark.sql("""
  select Education, count(*) as cnt from df group by Education order by Education desc
""").show()

+---------+---+
|Education|cnt|
+---------+---+
|        5| 48|
|        4|398|
|        3|572|
|        2|282|
|        1|170|
+---------+---+



In [110]:
df.show(3)

+---+-------------+--------------------+------+---------+---------+
|Age|MonthlyIncome|          Department|Gender|Education|Attrition|
+---+-------------+--------------------+------+---------+---------+
| 41|         5993|               Sales|Female|        2|      Yes|
| 49|         5130|Research & Develo...|  Male|        1|       No|
| 37|         2090|Research & Develo...|  Male|        2|      Yes|
+---+-------------+--------------------+------+---------+---------+
only showing top 3 rows



In [114]:
# 5) Процент увольнений (Attrition) по департаментам
# (Department, total_employees, attrition_count, attrition_rate)
spark.sql("""
  select
    Department,
    count(*) as cnt,

    sum(
      case
        when Attrition = 'Yes'
        then 1
        else 0
      end
    ) as attrition_count,

    avg(
      case
        when Attrition = 'No'
        then 1
        else 0
      end
    ) as attrition_rate

  from df
  group by Department
""").show(truncate=False)

+----------------------+---+---------------+------------------+
|Department            |cnt|attrition_count|attrition_rate    |
+----------------------+---+---------------+------------------+
|Sales                 |446|92             |0.7937219730941704|
|Research & Development|961|133            |0.8616024973985432|
|Human Resources       |63 |12             |0.8095238095238095|
+----------------------+---+---------------+------------------+



In [ ]:
# 6) Средний возраст сотрудников по гендеру (Gender, avg_age)
...

+------+-------+
|Gender|avg_age|
+------+-------+
|Female|   37.3|
|  Male|   36.7|
+------+-------+



In [ ]:
# 7) Соотношение мужчин и женщин в департаментах
# (Department, male_count, female_count)
...

+--------------------+----------+------------+
|          Department|male_count|female_count|
+--------------------+----------+------------+
|               Sales|       257|         189|
|Research & Develo...|       582|         379|
|     Human Resources|        43|          20|
+--------------------+----------+------------+



In [ ]:
# 8) Возрастающий тренд средней зарплаты в зависимости от возраста (Age, avg_income)
...

+---+----------+
|Age|avg_income|
+---+----------+
| 18|   1518.38|
| 19|    2075.0|
| 20|   2539.36|
| 21|   2491.46|
| 22|    2950.0|
| 23|   2515.36|
| 24|   3243.92|
| 25|   4144.77|
| 26|   3531.49|
| 27|   4190.42|
| 28|   3904.83|
| 29|   4584.41|
| 30|   4814.85|
| 31|   5525.39|
| 32|   5092.46|
| 33|    5500.5|
| 34|   5269.22|
| 35|   5425.28|
| 36|   5590.91|
| 37|   5853.86|
+---+----------+
only showing top 20 rows



### `Window function`

В SQL запросах часто встречаются оконные функции, которые необходимы при различных для аналитических задач с данными:

 - нарастающие итоги
 - скользящие средние
 - ранжирование


**Оконная функция в SQL** - функция, которая работает с выделенным набором строк (окном, партицией) и выполняет вычисление для этого набора строк в отдельном столбце.

##### `Сравнение оконной функции с групировкой`

![window_func](https://github.com/ekolmagorov/spark_hse_dpo/blob/ds-14/Lectures/Lecture4/images/window_func.png?raw=1)

В pyspark также есть возможность работы с оконными функциями

In [129]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, avg, rank, lag, lead, row_number

In [123]:
# Создание датафрейма
data = [
    (41, 5993, "Sales", "Female", 2, "Yes"),
    (49, 5130, "Research & Development", "Male", 1, "No"),
    (37, 2090, "Research & Development", "Male", 2, "Yes"),
    (33, 2909, "Research & Development", "Female", 4, "No")
]

columns = ["Age", "MonthlyIncome", "Department", "Gender", "Education", "Attrition"]
df = spark.createDataFrame(data, columns)

# Вывод результата
df.show()

+---+-------------+--------------------+------+---------+---------+
|Age|MonthlyIncome|          Department|Gender|Education|Attrition|
+---+-------------+--------------------+------+---------+---------+
| 41|         5993|               Sales|Female|        2|      Yes|
| 49|         5130|Research & Develo...|  Male|        1|       No|
| 37|         2090|Research & Develo...|  Male|        2|      Yes|
| 33|         2909|Research & Develo...|Female|        4|       No|
+---+-------------+--------------------+------+---------+---------+



In [126]:
# Определение окна по отделу
window_department = Window.partitionBy("Department")

# Задача 1: Средний доход по отделу
df = df.withColumn("AvgIncomeByDept", F.round(avg(col("MonthlyIncome")).over(window_department), 2))
df.show()

+---+-------------+--------------------+------+---------+---------+---------------+
|Age|MonthlyIncome|          Department|Gender|Education|Attrition|AvgIncomeByDept|
+---+-------------+--------------------+------+---------+---------+---------------+
| 49|         5130|Research & Develo...|  Male|        1|       No|        3376.33|
| 37|         2090|Research & Develo...|  Male|        2|      Yes|        3376.33|
| 33|         2909|Research & Develo...|Female|        4|       No|        3376.33|
| 41|         5993|               Sales|Female|        2|      Yes|         5993.0|
+---+-------------+--------------------+------+---------+---------+---------------+



In [128]:
# Задача 2: Разница между доходом сотрудника и средним доходом по отделу
df = df.withColumn("IncomeDifference", F.abs(col("MonthlyIncome") - col("AvgIncomeByDept")))
df.show()

+---+-------------+--------------------+------+---------+---------+---------------+-----------------+
|Age|MonthlyIncome|          Department|Gender|Education|Attrition|AvgIncomeByDept| IncomeDifference|
+---+-------------+--------------------+------+---------+---------+---------------+-----------------+
| 49|         5130|Research & Develo...|  Male|        1|       No|        3376.33|          1753.67|
| 37|         2090|Research & Develo...|  Male|        2|      Yes|        3376.33|          1286.33|
| 33|         2909|Research & Develo...|Female|        4|       No|        3376.33|467.3299999999999|
| 41|         5993|               Sales|Female|        2|      Yes|         5993.0|              0.0|
+---+-------------+--------------------+------+---------+---------+---------------+-----------------+



In [132]:
# Уникальные id сотрудника по департаменту
window_row_number = Window.partitionBy("Department").orderBy(col("MonthlyIncome").desc())

df = df.withColumn("IncomeRowNumber", row_number().over(window_row_number))
df.show()

+---+-------------+--------------------+------+---------+---------+---------------+-----------------+---------------+
|Age|MonthlyIncome|          Department|Gender|Education|Attrition|AvgIncomeByDept| IncomeDifference|IncomeRowNumber|
+---+-------------+--------------------+------+---------+---------+---------------+-----------------+---------------+
| 49|         5130|Research & Develo...|  Male|        1|       No|        3376.33|          1753.67|              1|
| 33|         2909|Research & Develo...|Female|        4|       No|        3376.33|467.3299999999999|              2|
| 37|         2090|Research & Develo...|  Male|        2|      Yes|        3376.33|          1286.33|              3|
| 41|         5993|               Sales|Female|        2|      Yes|         5993.0|              0.0|              1|
+---+-------------+--------------------+------+---------+---------+---------------+-----------------+---------------+



In [69]:
# Задача 3: Ранг дохода сотрудника внутри отдела (по убыванию)
window_rank = Window.partitionBy("Department").orderBy(col("MonthlyIncome").desc())
df = df.withColumn("IncomeRank", rank().over(window_rank))
df.show()

+---+-------------+--------------------+------+---------+---------+------------------+-------------------+----------+
|Age|MonthlyIncome|          Department|Gender|Education|Attrition|   AvgIncomeByDept|   IncomeDifference|IncomeRank|
+---+-------------+--------------------+------+---------+---------+------------------+-------------------+----------+
| 49|         5130|Research & Develo...|  Male|        1|       No|3376.3333333333335| 1753.6666666666665|         1|
| 33|         2909|Research & Develo...|Female|        4|       No|3376.3333333333335| -467.3333333333335|         2|
| 37|         2090|Research & Develo...|  Male|        2|      Yes|3376.3333333333335|-1286.3333333333335|         3|
| 41|         5993|               Sales|Female|        2|      Yes|            5993.0|                0.0|         1|
+---+-------------+--------------------+------+---------+---------+------------------+-------------------+----------+



In [133]:
# Задача 4: Предыдущее и следующее значение дохода сотрудника внутри отдела
df = df.withColumn("PrevIncome", lag("MonthlyIncome").over(window_rank))
df = df.withColumn("NextIncome", lead("MonthlyIncome").over(window_rank))

# Вывод результата
df.show()

+---+-------------+--------------------+------+---------+---------+---------------+-----------------+---------------+----------+----------+
|Age|MonthlyIncome|          Department|Gender|Education|Attrition|AvgIncomeByDept| IncomeDifference|IncomeRowNumber|PrevIncome|NextIncome|
+---+-------------+--------------------+------+---------+---------+---------------+-----------------+---------------+----------+----------+
| 49|         5130|Research & Develo...|  Male|        1|       No|        3376.33|          1753.67|              1|      NULL|      2909|
| 33|         2909|Research & Develo...|Female|        4|       No|        3376.33|467.3299999999999|              2|      5130|      2090|
| 37|         2090|Research & Develo...|  Male|        2|      Yes|        3376.33|          1286.33|              3|      2909|      NULL|
| 41|         5993|               Sales|Female|        2|      Yes|         5993.0|              0.0|              1|      NULL|      NULL|
+---+-------------+-

In [ ]:
# импортируем модуль, в котором располагаются оконные функции
from pyspark.sql.window import Window

In [ ]:
data = [
    ('James', 'Sales', 3000),
    ('Michael', 'Sales', 4600),
    ('Robert', 'Sales', 4100),
    ('Maria', 'Finance', 3000),
    ('Scott', 'Finance', 3300),
    ('Jen', 'Finance', 3900),
    ('Jeff', 'Marketing', 3000),
    ('Kumar', 'Marketing', 2000),
    ('Saif', 'Sales', 4100)
]

columns = ["emp_id", "dept_id", "salary"]
df = spark.createDataFrame(data=data, schema=columns)
df.printSchema(), df.show(truncate=False)

root
 |-- emp_id: string (nullable = true)
 |-- dept_id: string (nullable = true)
 |-- salary: long (nullable = true)

+-------+---------+------+
|emp_id |dept_id  |salary|
+-------+---------+------+
|James  |Sales    |3000  |
|Michael|Sales    |4600  |
|Robert |Sales    |4100  |
|Maria  |Finance  |3000  |
|Scott  |Finance  |3300  |
|Jen    |Finance  |3900  |
|Jeff   |Marketing|3000  |
|Kumar  |Marketing|2000  |
|Saif   |Sales    |4100  |
+-------+---------+------+



(None, None)

Cоздадим новую таблицу, в которой будет с агрегирована информация о средней зарплате в пределах департамента и объеденим её с таблицей с информацией о сотрудниках

In [ ]:
avg_salaries_df = df.groupBy(
    df.dept_id # по какому полю агрегируем
).agg(
    F.mean(df.salary # функция агрегации
).alias('avg_salary')) # задаём имя новой колнке

# объединяем с таблицей сотрудников
df.join(avg_salaries_df, on='dept_id', how='inner').show()

+---------+-------+------+----------+
|  dept_id| emp_id|salary|avg_salary|
+---------+-------+------+----------+
|    Sales|  James|  3000|    3950.0|
|    Sales|Michael|  4600|    3950.0|
|    Sales| Robert|  4100|    3950.0|
|  Finance|  Maria|  3000|    3400.0|
|    Sales|   Saif|  4100|    3950.0|
|  Finance|  Scott|  3300|    3400.0|
|  Finance|    Jen|  3900|    3400.0|
|Marketing|   Jeff|  3000|    2500.0|
|Marketing|  Kumar|  2000|    2500.0|
+---------+-------+------+----------+



In [ ]:
# создаём окно, в котором указываем по какому полю будет происходит создание окон
wspec = Window.partitionBy('dept_id')

(
    df
        .withColumn( # собираем зарплаты сотрудников в пределах своего департамента
            'salaries_list',
            F.collect_list(df.salary).over(wspec)
        )
        .withColumn(
            'avg_salary', # считаем среднюю зарплату в пределах департамента
            F.mean(df.salary).over(wspec)
        )
).show(truncate=False)

+-------+---------+------+------------------------+----------+
|emp_id |dept_id  |salary|salaries_list           |avg_salary|
+-------+---------+------+------------------------+----------+
|Maria  |Finance  |3000  |[3000, 3300, 3900]      |3400.0    |
|Scott  |Finance  |3300  |[3000, 3300, 3900]      |3400.0    |
|Jen    |Finance  |3900  |[3000, 3300, 3900]      |3400.0    |
|Jeff   |Marketing|3000  |[3000, 2000]            |2500.0    |
|Kumar  |Marketing|2000  |[3000, 2000]            |2500.0    |
|James  |Sales    |3000  |[3000, 4600, 4100, 4100]|3950.0    |
|Michael|Sales    |4600  |[3000, 4600, 4100, 4100]|3950.0    |
|Robert |Sales    |4100  |[3000, 4600, 4100, 4100]|3950.0    |
|Saif   |Sales    |4100  |[3000, 4600, 4100, 4100]|3950.0    |
+-------+---------+------+------------------------+----------+



Помимо указания полей, по которым будут созданы окна (partitionBy), можно ещё указать по какому полю должны быть упорядочены значения в пределах одного окна (orderBy). При добвалении порядка вычисление значений на основе оконной функции происходит последовательно, а не сразу.

In [ ]:
# создаём окна по полю dep_id и упорядочеваем значения в них на основе поля salary
wspec = Window.partitionBy('dept_id').orderBy('salary')
(
    df
        # последовательно собираем список зарплат сотрудников в одном департаменте
        .withColumn(
            'salaries_list',
            F.collect_list(df.salary).over(wspec)
        )
        # функцией row_number указываем порядок зарплаты сотрудника в пределах своего депратамента
        .withColumn(
            'row_number',
            F.row_number().over(wspec)
        )
        # последовательно считаем среднеюю зарплату сотрудников в пределах своего департамента
        .withColumn(
            'avg_salary',
            F.mean(df.salary).over(wspec)
        )
).show(truncate=True)

+-------+---------+------+--------------------+----------+------------------+
| emp_id|  dept_id|salary|       salaries_list|row_number|        avg_salary|
+-------+---------+------+--------------------+----------+------------------+
|  Maria|  Finance|  3000|              [3000]|         1|            3000.0|
|  Scott|  Finance|  3300|        [3000, 3300]|         2|            3150.0|
|    Jen|  Finance|  3900|  [3000, 3300, 3900]|         3|            3400.0|
|  Kumar|Marketing|  2000|              [2000]|         1|            2000.0|
|   Jeff|Marketing|  3000|        [2000, 3000]|         2|            2500.0|
|  James|    Sales|  3000|              [3000]|         1|            3000.0|
| Robert|    Sales|  4100|  [3000, 4100, 4100]|         2|3733.3333333333335|
|   Saif|    Sales|  4100|  [3000, 4100, 4100]|         3|3733.3333333333335|
|Michael|    Sales|  4600|[3000, 4100, 4100...|         4|            3950.0|
+-------+---------+------+--------------------+----------+------

Ещё наиболее частотные оконные функции:
 - lag(colname, N) - значение столбца сolname, который отстаёт от данного на N элементов
 - lag(colname, M) - значение столбца сolname, который опережает данный на M элементов

In [ ]:
wspec = Window.partitionBy('dept_id').orderBy('salary')
(
    df
        .withColumn('salaries_list', F.collect_list(df.salary).over(wspec))
        .withColumn('avg_salary', F.mean(df.salary).over(wspec))
        .withColumn('lag', F.lag(df.salary, 1).over(wspec))
        .withColumn('lead', F.lead(df.salary, 1).over(wspec))
).show(truncate=False)

+-------+---------+------+------------------------+------------------+----+----+
|emp_id |dept_id  |salary|salaries_list           |avg_salary        |lag |lead|
+-------+---------+------+------------------------+------------------+----+----+
|Maria  |Finance  |3000  |[3000]                  |3000.0            |NULL|3300|
|Scott  |Finance  |3300  |[3000, 3300]            |3150.0            |3000|3900|
|Jen    |Finance  |3900  |[3000, 3300, 3900]      |3400.0            |3300|NULL|
|Kumar  |Marketing|2000  |[2000]                  |2000.0            |NULL|3000|
|Jeff   |Marketing|3000  |[2000, 3000]            |2500.0            |2000|NULL|
|James  |Sales    |3000  |[3000]                  |3000.0            |NULL|4100|
|Robert |Sales    |4100  |[3000, 4100, 4100]      |3733.3333333333335|3000|4100|
|Saif   |Sales    |4100  |[3000, 4100, 4100]      |3733.3333333333335|4100|4600|
|Michael|Sales    |4600  |[3000, 4100, 4100, 4600]|3950.0            |4100|NULL|
+-------+---------+------+--

### `Join`

In [64]:
emp = [(1,"Smith",-1,"2018","10","M",3000), \
    (2,"Rose",1,"2010","20","M",4000), \
    (3,"Williams",1,"2010","10","M",1000), \
    (4,"Jones",2,"2005","10","F",2000), \
    (5,"Brown",2,"2010","40","",-1), \
      (6,"Brown",2,"2010","50","",-1) \
  ]
empColumns = ["emp_id","name","superior_emp_id","year_joined", \
       "emp_dept_id","gender","salary"]

empDF = spark.createDataFrame(data=emp, schema = empColumns)
empDF.printSchema()
empDF.show(truncate=False)

dept = [("Finance",10), \
    ("Marketing",20), \
    ("Sales",30), \
    ("IT",40) \
  ]
deptColumns = ["dept_name","dept_id"]
deptDF = spark.createDataFrame(data=dept, schema = deptColumns)
deptDF.printSchema()
deptDF.show(truncate=False)

root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- year_joined: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
|6     |Brown   |2              |2010       |50         |      |-1    |
+------+--------+---------------+-----------+-----------+------+-----

In [117]:
# Inner join
(
  empDF
  .join(
      deptDF,
      empDF.emp_dept_id == deptDF.dept_id,
      "inner"
  ).orderBy("emp_id").show(truncate=False)
)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [119]:
# Left outer join

(
  empDF
  .join(
      deptDF,
      empDF.emp_dept_id == deptDF.dept_id,
      "left"
  ).orderBy("emp_id").show(truncate=False)
)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
|6     |Brown   |2              |2010       |50         |      |-1    |NULL     |NULL   |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [120]:
# Right outer join
(
  empDF
  .join(
      deptDF,
      empDF.emp_dept_id ==  deptDF.dept_id,
      "right"
  ).orderBy("emp_id").show(truncate=False)
)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|NULL  |NULL    |NULL           |NULL       |NULL       |NULL  |NULL  |Sales    |30     |
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [121]:
# Full outer join
(
  empDF
  .join(
      deptDF,
      empDF.emp_dept_id == deptDF.dept_id,
      "full"
  ).orderBy("emp_id").show(truncate=False)
)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|NULL  |NULL    |NULL           |NULL       |NULL       |NULL  |NULL  |Sales    |30     |
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
|6     |Brown   |2              |2010       |50         |      |-1    |NULL     |NULL   |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



In [ ]:
# Left semi join
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"leftsemi") \
   .show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
+------+--------+---------------+-----------+-----------+------+------+



In [ ]:
# Left anti join
empDF.join(deptDF,empDF.emp_dept_id ==  deptDF.dept_id,"leftanti") \
   .show(truncate=False)

+------+-----+---------------+-----------+-----------+------+------+
|emp_id|name |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+-----+---------------+-----------+-----------+------+------+
|6     |Brown|2              |2010       |50         |      |-1    |
+------+-----+---------------+-----------+-----------+------+------+



### `UDF`

В предыдущих примеров, при работе с элементами таблиц использовались встроенные методы обработки строк spark таблиц из модуля pyspark.sql.functions. Но фрейморк не запрещает определять свои кастомные функции обработки.

In [ ]:
import pandas as pd

@F.pandas_udf('double', F.PandasUDFType.SCALAR)
def add_one(v):
    return v + 1

ratings_df.select(add_one(ratings_df.rating)).show()
ratings_df.withColumn(
    'plus_one', add_one(ratings_df.rating)
).show()

/usr/local/lib/python3.11/dist-packages/pyspark/sql/pandas/functions.py:407: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


NameError: name 'ratings_df' is not defined

Также можно определить свои агрегирующие функции

In [ ]:
@F.pandas_udf(ratings_df.schema, F.PandasUDFType.GROUPED_MAP)
# Input/output are both a pandas.DataFrame
def subtract_mean(pdf):
    return pdf.assign(rating=pdf.rating - pdf.rating.mean())

ratings_df.groupby('movie_id').apply(subtract_mean).show()

/usr/local/lib/python3.10/site-packages/pyspark/sql/pandas/group_ops.py:98: UserWarning: It is preferred to use 'applyInPandas' over this API. This API will be deprecated in the future releases. See SPARK-28264 for more details.
  warnings.warn(


+-------+--------+------------+---------+
|user_id|movie_id|      rating|timestamp|
+-------+--------+------------+---------+
|    181|      31|   0.8865249|977087101|
|    195|      31|-0.113475084|991013952|
|    203|      31|   -2.113475|976929358|
|    223|      31|-0.113475084|976905652|
|    268|      31|   0.8865249|976647137|
|    368|      31|-0.113475084|976670975|
|    517|      31|   0.8865249|976204301|
|    524|      31|   -2.113475|976171096|
|    528|      31|   1.8865249|980039160|
|    531|      31|  -1.1134751|978973034|
|    536|      31|-0.113475084|976137228|
|    543|      31|   0.8865249|976159357|
|    616|      31|-0.113475084|975802599|
|    676|      31|   0.8865249|975684957|
|    678|      31|   0.8865249|989241973|
|    692|      31|-0.113475084|978375055|
|    699|      31|-0.113475084|975563262|
|    710|      31|-0.113475084|978586309|
|    752|      31|   -2.113475|975461295|
|    777|      31|-0.113475084|975520841|
+-------+--------+------------+---

#### `Формат Parquet`

Обработка больших данных увеличивает нагрузку на подсистему хранения — Hadoop хранит данные избыточно для достижения отказоустойчивости. Кроме дисков, нагружаются процессор, сеть, система ввода-вывода и так далее. По мере роста объема данных увеличивается и стоимость их обработки и хранения.

Различные форматы файлов в Hadoop придуманы для решения именно этих проблем. Выбор подходящего формата файла может дать некоторые существенные преимущества:

 - Более быстрое время чтения.
 - Более быстрое время записи.
 - Разделяемые файлы.
 - Поддержка эволюции схем.
 - Расширенная поддержка сжатия.

![parquet](https://github.com/ekolmagorov/spark_hse_dpo/blob/ds-14/Lectures/Lecture4/images/data_sizes.png?raw=1)

Parquet - это столбчатый формат хранения данных. Это помогает повысить производительность,
иногда значительно, разрешая хранение и доступ к данным для каждого столбца&

Столбчатый формат более эффективен, когда вам нужно запросить из таблицы несколько столбцов. Он прочитает только необходимые столбцы, потому что они находятся по соседству. Таким образом, операции ввода-вывода сводятся к минимуму.

![parquet_columns](https://github.com/ekolmagorov/spark_hse_dpo/blob/ds-14/Lectures/Lecture4/images/parquet_columns.png?raw=1)

Сохраним датафрейм ratings_df в формате Parquet и посмотрим его содержимое.

In [134]:
hr_data = spark.read.csv("hr_data.csv", header=True, inferSchema=True)

In [135]:
hr_data.show(3)

+---+-------------+--------------------+------+---------+---------+
|Age|MonthlyIncome|          Department|Gender|Education|Attrition|
+---+-------------+--------------------+------+---------+---------+
| 41|         5993|               Sales|Female|        2|      Yes|
| 49|         5130|Research & Develo...|  Male|        1|       No|
| 37|         2090|Research & Develo...|  Male|        2|      Yes|
+---+-------------+--------------------+------+---------+---------+
only showing top 3 rows



In [136]:
hr_data.write.mode("overwrite").parquet("hr_data_parquet")

In [137]:
!du -h /content/*

56K	/content/hr_data.csv
24K	/content/hr_data_parquet
55M	/content/sample_data
4.0K	/content/spark-warehouse


In [138]:
hr_data_p = spark.read.parquet("hr_data_parquet")

In [140]:
hr_data_p.show(5)

+---+-------------+--------------------+------+---------+---------+
|Age|MonthlyIncome|          Department|Gender|Education|Attrition|
+---+-------------+--------------------+------+---------+---------+
| 41|         5993|               Sales|Female|        2|      Yes|
| 49|         5130|Research & Develo...|  Male|        1|       No|
| 37|         2090|Research & Develo...|  Male|        2|      Yes|
| 33|         2909|Research & Develo...|Female|        4|       No|
| 27|         3468|Research & Develo...|  Male|        1|       No|
+---+-------------+--------------------+------+---------+---------+
only showing top 5 rows



In [141]:
hr_data_p.printSchema()

root
 |-- Age: integer (nullable = true)
 |-- MonthlyIncome: integer (nullable = true)
 |-- Department: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Education: integer (nullable = true)
 |-- Attrition: string (nullable = true)



После процедуры сохранения файла можно увидеть два файла в формате parquet. Количество сохранённых файлов равно количеству партиций RDD, которые были у данного датафрейма

In [89]:
# выведем кол-во партиций
hr_data.rdd.getNumPartitions()

1

Sprk позволяет при записи создавать разбиение данных на основе значений некоторого столбца. Таким образом каждое уникальное значение в нём создаст свою партцию.

Зачастую это нужно для эффективной работы с нужной порцией данных, например, когда нужно обработать данные в одной партииции.

In [ ]:
# сохраним данные и разобъём их на партции по полю Department
hr_data.write.partitionBy("Department").mode("overwrite").parquet("hr_dep")

Теперь если зайти в директорию hr_dep, то можно увидедеть, что spark создал подпапки с названиями пратиций. Каждая папка хранит данные только своей партиции

### `Pivot/Unpivot`

Скачаем модельные данные и распакуем их в директорию ml-1m

In [ ]:
! wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
! unzip -o ml-1m.zip

--2023-10-27 22:40:37--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.2’

ml-1m.zip.2         100%[===================>]   5.64M  3.91MB/s    in 1.4s    

2023-10-27 22:40:40 (3.91 MB/s) - ‘ml-1m.zip.2’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [ ]:
# указываем схему данных для первой таблицы
schema = (
    T.StructType()
        .add('movie_id', T.IntegerType()) # колонка movie_id будет иметь целочиселнный тип
        .add('movie', T.StringType()) # колонка movie будет иметь строковый
        .add('categories', T.StringType()) # и колонка categories также будет строкой
)
# загружаем в spark данные из первой таблицы
movies_df = (
    spark.read
      .format('csv') # указываем формат считываем данных
      .option("header", False) # считываем данные без заголовка
      .option("sep", '::') # в качестве разделителя столбцов указываем ::
      .schema(schema) # в качестве схемы указываем описанную выше схему
      .load('./ml-1m/movies.dat') # указываем источник данных
)

# тоже самое только для второй таблицы
schema = (
    T.StructType()
        .add('user_id', T.IntegerType())
        .add('movie_id', T.IntegerType())
        .add('rating', T.FloatType()) # данный тип - вещественные числа
        .add('timestamp', T.StringType())
)
ratings_df = (
    spark.read.format('csv')
      .option("header", False)
      .option("sep", '::')
      .schema(schema)
      .load('./ml-1m/ratings.dat')
)

# тоже самое только для третьей таблицы
schema = (
    T.StructType()
        .add('user_id', T.IntegerType())
        .add('gender', T.StringType())
        .add('age', T.IntegerType())
        .add('occupation', T.IntegerType())
        .add('zip-code', T.StringType())
)
users_df = (
    spark.read.format('csv')
      .option("header", False)
      .option("sep", '::')
      .schema(schema)
      .load('./ml-1m/users.dat')
)

Во многих задачах зачастую встречается необходимость сформировать колонки на основе данных, хранящихся в другом столбце. То есть требуется перевести таблицу в wide формат.

Например, данная операция нужна была во второй лекции, когда для значения класса нужно было получить one-hot представление

##### `Визуализация того, как это устроено в pandas`

![pandas-pivot](https://github.com/ekolmagorov/spark_hse_dpo/blob/ds-14/Lectures/Lecture4/images/pandas-dataframe-pivot-1.svg?raw=1)

Spark также поддерживает данную операцию, но делается это вызовом не одного метода, а нескольких последовательных:

groupBy -> pivot -> agg

In [ ]:
ratings_df.show()

+-------+--------+------+---------+
|user_id|movie_id|rating|timestamp|
+-------+--------+------+---------+
|      1|    1193|   5.0|978300760|
|      1|     661|   3.0|978302109|
|      1|     914|   3.0|978301968|
|      1|    3408|   4.0|978300275|
|      1|    2355|   5.0|978824291|
|      1|    1197|   3.0|978302268|
|      1|    1287|   5.0|978302039|
|      1|    2804|   5.0|978300719|
|      1|     594|   4.0|978302268|
|      1|     919|   4.0|978301368|
|      1|     595|   5.0|978824268|
|      1|     938|   4.0|978301752|
|      1|    2398|   4.0|978302281|
|      1|    2918|   4.0|978302124|
|      1|    1035|   5.0|978301753|
|      1|    2791|   4.0|978302188|
|      1|    2687|   3.0|978824268|
|      1|    2018|   4.0|978301777|
|      1|    3105|   5.0|978301713|
|      1|    2797|   4.0|978302039|
+-------+--------+------+---------+
only showing top 20 rows



In [ ]:
pivot_df = (
    ratings_df
        .groupBy(ratings_df.user_id)
        .pivot('movie_id')
        .agg(F.first(ratings_df.rating))
)
pivot_df.where(pivot_df.user_id < 4).toPandas()

23/10/27 22:40:49 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
23/10/27 22:40:59 WARN DAGScheduler: Broadcasting large task binary with size 2.4 MiB                                                                                    (0 + 1) / 1]


,user_id,1,2,3,4,5,6,7,8,9,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
top_movies_df = ratings_df.groupBy(
    ratings_df.movie_id # групируем по movie_id
).agg(
    F.count(ratings_df.rating).alias('rates'), # создаём колонку rates c количество оценок
    F.mean(ratings_df.rating).alias('avg_rating') # создаём колонку avg_rating с средней оценкой
).sort(
    'rates', ascending=False # упорядочиваем фильмы по количеству оценок в убывающем порядке
).limit(100) # берём только 100 самых оцениваемых фильмов

top_movies_df.show()

+--------+-----+------------------+
|movie_id|rates|        avg_rating|
+--------+-----+------------------+
|    2858| 3428|4.3173862310385065|
|     260| 2991| 4.453694416583082|
|    1196| 2990| 4.292976588628763|
|    1210| 2883| 4.022892819979188|
|     480| 2672|3.7638473053892216|
|    2028| 2653| 4.337353938937053|
|     589| 2649| 4.058512646281616|
|    2571| 2590| 4.315830115830116|
|    1270| 2583|3.9903213317847466|
|     593| 2578|4.3518231186966645|
|    1580| 2538| 3.739952718676123|
|    1198| 2514| 4.477724741447892|
|     608| 2513| 4.254675686430561|
|    2762| 2459| 4.406262708418057|
|     110| 2443| 4.234957020057307|
|    2396| 2369| 4.127479949345715|
|    1197| 2318|4.3037100949094045|
|     527| 2304| 4.510416666666667|
|    1617| 2288| 4.219405594405594|
|    1265| 2278| 3.953028972783143|
+--------+-----+------------------+
only showing top 20 rows



In [ ]:
top_movies = top_movies_df.rdd.map(lambda x: x.movie_id).collect()

In [ ]:
pivot_top_df = (
    ratings_df
        .groupBy(ratings_df.user_id)
        .pivot('movie_id', top_movies)
        .agg(F.first(ratings_df.rating))
)
pivot_top_df.where(pivot_df.user_id < 3).toPandas()

,user_id,2858,260,1196,1210,480,2028,589,2571,1270,...,750,2699,39,21,1393,2804,588,2406,1220,733
0,1,NaN,4.0,NaN,NaN,NaN,5.0,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN
1,2,4.0,NaN,5.0,4.0,5.0,4.0,4.0,4.0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


В качестве второго аргумента в методе pivot можно указать подмножество допустимых значений на основе, которых сформируются новые столбцы

In [ ]:
# формируем таблицу с столбцы только по top-100 самых оцениваемых фильмов
pivot_top_df = (
    ratings_df
        .groupBy(ratings_df.user_id)
        .pivot('movie_id', top_movies) # вторым параметром ограничиваем дотупстимые столбцы
        .agg(F.first(ratings_df.rating))
)
pivot_top_df.where(pivot_top_df.user_id < 4).toPandas()

,user_id,2858,260,1196,1210,480,2028,589,2571,1270,...,750,2699,39,21,1393,2804,588,2406,1220,733
0,1,NaN,4.0,NaN,NaN,NaN,5.0,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,NaN,NaN
1,3,4.0,5.0,4.0,4.0,4.0,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
2,2,4.0,NaN,5.0,4.0,5.0,4.0,4.0,4.0,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


Как можно заметить кол-во пустых столбцов стало гораздо меньше

Скачаем данные из предыдущей лекции, с которыми будем в дальнейшем работать

In [ ]:
import json
import requests
import subprocess
import zipfile

folder_url = 'https://disk.yandex.lt/d/JnDy1h48pJI7IA'
file_url = '/m5-forecating-accuracy.zip'
# запрос ссылки на скачивание
response = requests.get('https://cloud-api.yandex.net/v1/disk/public/resources/download',
                 params={'public_key': folder_url, 'path': file_url})
# 'парсинг' ссылки на скачивание
data_link = response.json()['href']

filename = 'm5-forecating-accuracy.zip'
path = "./m5-forecasting-accuracy"

# запускаем скачивание вызовом команды wget из python
subprocess.run(
    ['wget', '-O', filename, data_link], # команда для исполнения
    stdout=subprocess.DEVNULL, # убираем печать отладочной информации
    stderr=subprocess.STDOUT
)

#распакуем данные по пути path
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [ ]:
# считаем данные
df_validation = (
    spark.read.format('csv')
      .option("inferSchema", True) # указываем spark самому определить тип данных
      .option("header", True)
      .option("sep", ',')
      .load(f"{path}/sales_train_validation.csv")
)
df_validation.limit(10).toPandas()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
5,HOBBIES_1_006_CA_1_validation,HOBBIES_1_006,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,0,1,0,0,0,2,0,0
6,HOBBIES_1_007_CA_1_validation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,1,0,1,0,0,1,1
7,HOBBIES_1_008_CA_1_validation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,12,15,0,0,...,0,0,1,37,3,4,6,3,2,1
8,HOBBIES_1_009_CA_1_validation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,2,0,7,3,...,0,0,1,1,6,0,0,0,0,0
9,HOBBIES_1_010_CA_1_validation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,0,0,1,0,...,1,0,0,0,0,0,0,2,0,2


Для операции pivot существует противоположная операция "вытягивания" таблицы в длину на основе значений из столбцов.

Для этого применяется оперция stack:
```SQL
stack(
    N, new_value_name_1, old_colname_1, ..., new_value_name_N, old_colname_N
) as (new_colname, value_colname)
```

In [ ]:
unpivot_expression = "stack(2, 'd_1_new', d_1, 'd_2', d_2) as (d, sales)"
unpivot_df = (
    df_validation
        .select('id', F.expr(unpivot_expression))
)

unpivot_df.show(truncate=False)

+-----------------------------+-------+-----+
|id                           |d      |sales|
+-----------------------------+-------+-----+
|HOBBIES_1_001_CA_1_validation|d_1_new|0    |
|HOBBIES_1_001_CA_1_validation|d_2    |0    |
|HOBBIES_1_002_CA_1_validation|d_1_new|0    |
|HOBBIES_1_002_CA_1_validation|d_2    |0    |
|HOBBIES_1_003_CA_1_validation|d_1_new|0    |
|HOBBIES_1_003_CA_1_validation|d_2    |0    |
|HOBBIES_1_004_CA_1_validation|d_1_new|0    |
|HOBBIES_1_004_CA_1_validation|d_2    |0    |
|HOBBIES_1_005_CA_1_validation|d_1_new|0    |
|HOBBIES_1_005_CA_1_validation|d_2    |0    |
|HOBBIES_1_006_CA_1_validation|d_1_new|0    |
|HOBBIES_1_006_CA_1_validation|d_2    |0    |
|HOBBIES_1_007_CA_1_validation|d_1_new|0    |
|HOBBIES_1_007_CA_1_validation|d_2    |0    |
|HOBBIES_1_008_CA_1_validation|d_1_new|12   |
|HOBBIES_1_008_CA_1_validation|d_2    |15   |
|HOBBIES_1_009_CA_1_validation|d_1_new|2    |
|HOBBIES_1_009_CA_1_validation|d_2    |0    |
|HOBBIES_1_010_CA_1_validation|d_1

Теперь произвём обратную оперцию "расширения" (pivot) для новой таблицы

In [ ]:
(
    unpivot_df
        .groupBy(unpivot_df.id)
        .pivot('d')
        .agg(F.sum(unpivot_df.sales))
).show(truncate=False)

+-------------------------------+-------+---+
|id                             |d_1_new|d_2|
+-------------------------------+-------+---+
|HOBBIES_1_304_CA_4_validation  |0      |0  |
|FOODS_3_257_CA_3_validation    |1      |0  |
|HOUSEHOLD_1_157_CA_2_validation|0      |0  |
|FOODS_2_387_CA_1_validation    |0      |0  |
|FOODS_2_225_CA_3_validation    |4      |6  |
|FOODS_2_375_CA_2_validation    |0      |0  |
|FOODS_2_354_CA_4_validation    |0      |0  |
|FOODS_3_297_TX_1_validation    |0      |0  |
|HOUSEHOLD_1_319_CA_3_validation|1      |1  |
|HOUSEHOLD_2_090_CA_4_validation|0      |0  |
|FOODS_3_568_CA_4_validation    |0      |0  |
|HOBBIES_1_364_TX_2_validation  |3      |0  |
|FOODS_1_101_CA_2_validation    |0      |0  |
|FOODS_3_558_TX_1_validation    |1      |2  |
|FOODS_2_011_CA_1_validation    |1      |1  |
|FOODS_3_035_CA_3_validation    |3      |2  |
|FOODS_2_174_CA_2_validation    |0      |0  |
|HOBBIES_1_396_CA_4_validation  |0      |0  |
|HOBBIES_2_044_CA_2_validation  |0

Проверим, что в ходе операций unpivot->pivot данные остались такими же какими они и были изначально, взяв последнюю строку из таблицы выше

In [ ]:
(
    df_validation
        .where(df_validation.id == 'HOBBIES_1_320_CA_3_validation')
        .select('id', 'd_1', 'd_2')
).show(truncate=False)

+-----------------------------+---+---+
|id                           |d_1|d_2|
+-----------------------------+---+---+
|HOBBIES_1_320_CA_3_validation|7  |2  |
+-----------------------------+---+---+

